# Coral Bleaching
## Predicting the Likelihood and Severity of Coral Reef Bleaching Events Based on Various Environmental Factors

Authors: Jaimie Chin & Maro Aboelwafa  
Course: DS.UA.301 - Advanced Topics in Data Science: Machine Learning for Climate Change  
Date: 22 March 2023  

## Background
The climate issue we are tackling is predicting the likelihood and severity of coral reef bleaching events based on various environmental factors. Coral reefs are sensitive to environmental changes such as temperature, salinity, nutrient levels and water acidity. When these factors reach certain thresholds, they  trigger coral bleaching, a process in which the coral expels the symbiotic algae that live on it, causing the coral to turn white and possibly die.

To address this, we aim to use machine learning to analyze datasets of environmental data, such as water temperature to develop algorithms that can predict the likelihood and severity of coral bleaching events.

### Data 
We will be using the Bleaching and environmental data for global coral reef sites (1980-2020) from the Biological & Chemical Oceanography Data Management Office. The data includes information on the presence and absence of coral bleaching, allowing comparative analyses and the determination of geographical bleaching thresholds, together with site exposure, distance to land, mean turbidity, cyclone frequency, and a suite of sea-surface temperature metrics at the times of survey. 

Data Server: [Bleaching and Environmental Data](http://dmoserv3.whoi.edu/jg/info/BCO-DMO/Coral_Reef_Brightspots/bleaching_and_env_data%7Bdir=dmoserv3.whoi.edu/jg/dir/BCO-DMO/Coral_Reef_Brightspots/,data=dmoserv3.bco-dmo.org:80/jg/serv/BCO-DMO/Coral_Reef_Brightspots/global_bleaching_environmental.brev0%7D?)


## Import Packages

In [2]:
# Import packages & libraries 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load Dataset 

In [3]:
# Import Global Bleaching & Environmental Data 
data_path = "data/global_bleaching_environmental.csv"
types = {'Distance_to_Shore': float, 'Turbidity': float, 'Percent_Bleaching': float}
bleach_df = pd.read_csv(data_path, sep='\s*[,]\s*', engine='python', na_values=['nd'], dtype=types)

In [4]:
# View sample of the data 
bleach_df.sample(5)

,Site_ID,Sample_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Reef_ID,Realm_Name,Ecoregion_Name,Country_Name,...,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,Date,Site_Comments,Sample_Comments,Bleaching_Comments
22223,999450,10330577,AGRRA,24.7126,-77.7477,Atlantic,NaN,Tropical Atlantic,Bahamas and Florida Keys,Bahamas,...,5.0,0.0,0.00,0.86,6.42,0.23,2002-04-22,NaN,NaN,NaN
34462,5686,10307930,Reef_Check,5.7539,103.0289,Pacific,103.1.44.07E.5.45.14.03N,Central Indo-Pacific,Sunda Shelf south-east Asia,Malaysia,...,9.5,1.0,1.08,1.20,9.86,0.40,2019-06-19,NaN,NaN,NaN
29902,1143,10313585,Reef_Check,-14.9302,145.6754,Pacific,145.40.523E.14.55.812S,Central Indo-Pacific,Central and northern Great Barrier Reef,Australia,...,5.0,1.0,1.06,1.21,7.71,0.48,2004-03-03,NaN,NaN,NaN
509,9127,10328458,Donner,24.9860,-80.4150,Atlantic,NaN,Tropical Atlantic,Bahamas and Florida Keys,United States,...,5.0,0.0,0.00,0.77,5.73,0.22,2005-09-15,NaN,NaN,NaN
17997,6922,10318228,Reef_Check,26.3739,56.4151,Arabian Gulf,56.24.54.28E.26.22.36.03N,Western Indo-Pacific,Gulf of Oman,Oman,...,6.0,1.0,1.88,1.60,13.18,0.47,2012-10-11,NaN,NaN,NaN


## Data Cleaning & Exploration
* Attribute Information & Null values 
* Distributions of each attribute 
* Visualizations
* Correlation Matrix 


In [5]:
bleach_df.shape

(41361, 62)

## Feature Selection
* Adding, Dropping Features of Importance 
* Encoding Features 
* Discard Unnecessary Features 
* Impute missing information if necessary

## Model Implementation
* Random Forest
* Gradient Boosting 
* ANN

## Evaluation & Analysis 
* Accuracy
* Precision
* Recall
* F1 score 

## Optimization
* Cross-Validation
* Grid-Search

## Final Evaluation & Analysis
* Accuracy
* Precision
* Recall
* F1 score 

## Final Visualizations 
* Plot coral bleaching severity?